In [1]:
from Imitation_Learning.Training.RGB.config import Config
from Imitation_Learning.Training.RGB.single_view_generator import DataLoader
import sys
import tqdm
import numpy as np
import random
if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

if '/media/eslam/426b7820-cb81-4c46-9430-be5429970ddb/home/eslam/Future_Imitiation/video_prediction-master/' not in sys.path:
    sys.path.append('/media/eslam/426b7820-cb81-4c46-9430-be5429970ddb/home/eslam/Future_Imitiation/video_prediction-master/')
from scripts.generate_future import inference_future_generator

config = Config()
future_generator = inference_future_generator(config.f_stacking_frames, batch_size=200)
data_loader = DataLoader(config, future_generator, is_training=False)

for itr in range(data_loader.num_of_episodes):
    print(itr)
    measurement = np.load(data_loader.config.labels_dir + '/' + data_loader.measurements_files[itr])
    # Generate list of random numbers without repetition
    item_num = random.sample(range(len(measurement)), len(measurement))
    x_s, y, direction = data_loader.get_batch(episode_num=itr, item_nums=item_num)
    
    temp = []
    stacked_batch = []
    temp_gray = []
    for num in range(len(x_s)):
        temp.append(x_s[num])
        if (num + 1) % (config.p_stacking_frames + config.f_stacking_frames) == 0:
            for k in range((config.p_stacking_frames + config.f_stacking_frames)):
                temp_gray.append(cv2.cvtColor(temp[k], cv2.COLOR_BGR2GRAY))
            stacked_batch.append(
                np.swapaxes(np.swapaxes(np.asarray(temp_gray), 0, 1), 1, 2))
            temp = []
            temp_gray = []
    input_images = stacked_batch
    # shape here is (200,200,200,8)
    rgb_saving_dir = "/media/eslam/0d208863-5cdb-4a43-9794-3ca8726831b3/preprocessed_future_data/rgb/Val/"
    direction_saving_dir = "/media/eslam/0d208863-5cdb-4a43-9794-3ca8726831b3/preprocessed_future_data/direction/Val/"
    labels_saving_dir = "/media/eslam/0d208863-5cdb-4a43-9794-3ca8726831b3/preprocessed_future_data/labels/Val/"
    np.save(rgb_saving_dir+"Episode_"+str(itr).zfill(5)+".npy", np.array(input_images, dtype=np.uint8))
    np.save(direction_saving_dir+"Episode_"+str(itr).zfill(5)+".npy", direction)
    np.save(labels_saving_dir+"Episode_"+str(itr).zfill(5)+".npy", y)


/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

creating restore saver from checkpoint /media/eslam/426b7820-cb81-4c46-9430-be5429970ddb/home/eslam/Future_Imitiation/video_prediction-master/logs/carla_intel/ours_savp
checkpoint variables that were not used for restoring because they are not in the graph:
     d_apply_gradients/beta1_power
     d_apply_gradients/beta2_power
     discriminator/encoder/video/sn_conv0_0/bias
     discriminator/encoder/video/sn_conv0_0/bias/Adam
     discriminator/encoder/video/sn_conv0_0/bias/Adam_1
     discriminator/encoder/video/sn_conv0_0/conv3d/kernel
     discriminator/encoder/video/sn_conv0_0/conv3d/kernel/Adam
     discriminator/encoder/video/sn_conv0_0/conv3d/kernel/Adam_1
     discriminator/encoder/video/sn_conv0_0/conv3d/u
     discriminator/encoder/video/sn_conv0_1/bias
     discriminator/encoder/video/sn_conv0_1/bias/Adam
     discriminator/encoder/video/sn_conv0_1/bias/Adam_1
     discriminator/encoder/video/sn_conv0_1/conv3d/kernel
     discriminator/encoder/video/sn_conv0_1/conv3d/kernel

Loading labels done 
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226


In [2]:
print(x_s[5].shape)
print(len(x_s))

(200, 200, 3)
1600


In [6]:
input_images[0].shape

(200, 200, 8)

200